In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [2]:
from shutil import copy2

In [3]:
import matplotlib.pyplot as plt

In [4]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
labels = []
dist = []
for race in races:
    for gender in genders:
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [5]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.3287125527858734


In [6]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
accuracies = {}
for race in races:
    for gender in genders:
        accuracies[f"{race}_{gender}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TN'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FN'] += 1
            else:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TP'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_{gender}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [7]:
print(accuracies)

{'African_Man': {'TP': 2825, 'TN': 114, 'FP': 115, 'FN': 2825}, 'African_Woman': {'TP': 53, 'TN': 8, 'FP': 1, 'FN': 59}, 'Asian_Man': {'TP': 2049, 'TN': 79, 'FP': 78, 'FN': 2053}, 'Asian_Woman': {'TP': 808, 'TN': 64, 'FP': 26, 'FN': 843}, 'Caucasian_Man': {'TP': 2188, 'TN': 65, 'FP': 2, 'FN': 2250}, 'Caucasian_Woman': {'TP': 717, 'TN': 30, 'FP': 2, 'FN': 746}, 'Indian_Man': {'TP': 2245, 'TN': 70, 'FP': 61, 'FN': 2256}, 'Indian_Woman': {'TP': 657, 'TN': 28, 'FP': 15, 'FN': 667}}


In [8]:
gar = {}
far = {}
scores = {}
for race in races:
    for gender in genders:
        tp = accuracies[f"{race}_{gender}"]['TP']
        tn = accuracies[f"{race}_{gender}"]['TN']
        fp = accuracies[f"{race}_{gender}"]['FP']
        fn = accuracies[f"{race}_{gender}"]['FN']
        gar[f"{race}_{gender}"] = tp/(tn+tp)
        far[f"{race}_{gender}"] = fp/(fn+fp)
        scores[f"{race}_{gender}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [9]:
gar

{'African_Man': 0.9612112963593059,
 'African_Woman': 0.8688524590163934,
 'Asian_Man': 0.962875939849624,
 'Asian_Woman': 0.926605504587156,
 'Caucasian_Man': 0.9711495783399912,
 'Caucasian_Woman': 0.9598393574297188,
 'Indian_Man': 0.9697624190064795,
 'Indian_Woman': 0.9591240875912409}

In [10]:
far

{'African_Man': 0.0391156462585034,
 'African_Woman': 0.016666666666666666,
 'Asian_Man': 0.03660253402158611,
 'Asian_Woman': 0.02991944764096663,
 'Caucasian_Man': 0.0008880994671403197,
 'Caucasian_Woman': 0.00267379679144385,
 'Indian_Man': 0.026327147173068624,
 'Indian_Woman': 0.021994134897360705}

In [11]:
scores

{'African_Man': 96.10477972444293,
 'African_Woman': 92.56198347107438,
 'Asian_Man': 96.31368865930969,
 'Asian_Woman': 94.8305571510626,
 'Caucasian_Man': 98.51276359600443,
 'Caucasian_Woman': 97.85953177257525,
 'Indian_Man': 97.17184801381693,
 'Indian_Woman': 96.85442574981712}